In [3]:
# Import library
import pandas as pd  
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.metrics.pairwise import cosine_similarity  

In [8]:
# Read dataset  
df = pd.read_csv('Dataset/ShopperSentiments.csv',encoding='ISO-8859-1') 
df.tail(10) 

,reviewer_id,store_location,latitude,longitude,date,month,year,title,review,review-label
278090,278090.0,CA,56.130366,-106.346771,2018,4,2027 00:00:00,Always great products,Always great products. Fantastic customer serv...,5
278091,278091.0,US,37.090240,-95.712891,2018,4,2027 00:00:00,Tons of Compliments,Tees are great quality and they were a big hit...,5
278092,278092.0,DE,51.165691,10.451526,2018,4,2027 00:00:00,got no product that i buy till now,"got no product that i buy till now, no shipmen...",1
278093,278093.0,US,37.090240,-95.712891,2018,4,2027 00:00:00,These shirts literally started peelingï¿½ï¿½ï¿½,These shirts literally started peeling after i...,2
278094,278094.0,US,37.090240,-95.712891,2018,4,2027 00:00:00,Great site,Great site. Supports independent artists while...,5
278095,278095.0,US,37.090240,-95.712891,2018,4,2027 00:00:00,Highly recommend!,"Delivery is quick, arrives when promised. Shir...",5
278096,278096.0,HR,45.100000,15.200000,2018,4,2027 00:00:00,Great quality,"Great quality, fast delivery, would recommend!",5
278097,278097.0,US,37.090240,-95.712891,2018,4,2027 00:00:00,Dudes rock.,Dudes rock. Seriously.,5
278098,278098.0,US,37.090240,-95.712891,2018,4,2027 00:00:00,Shipping was fast the T-shirt was justï¿½ï¿½ï¿½,Shipping was fast the T-shirt was just right g...,5
278099,NaN,US,37.090240,-95.712891,2018,4,2027 00:00:00,Not great quality,Print of t shirt was blurry and appeared faded...,2


# Data Cleaning

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278100 entries, 0 to 278099
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   reviewer_id     278099 non-null  float64
 1   store_location  278100 non-null  object 
 2   latitude        278100 non-null  float64
 3   longitude       278100 non-null  float64
 4   date            278100 non-null  int64  
 5   month           278100 non-null  int64  
 6   year            278100 non-null  object 
 7   title           278088 non-null  object 
 8   review          247597 non-null  object 
 9   review-label    278100 non-null  int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 21.2+ MB


In [5]:
df.isna().sum()

reviewer_id           1
store_location        0
latitude              0
longitude             0
date                  0
month                 0
year                  0
title                12
review            30503
review-label          0
dtype: int64

In [5]:
# Content-Based Filtering  
tfidf = TfidfVectorizer(stop_words='english')  
tfidf_matrix = tfidf.fit_transform(df['review'])  
cosine_sim_cbf = cosine_similarity(tfidf_matrix, tfidf_matrix)  

ValueError: np.nan is an invalid document, expected byte or unicode string.